# Проект для «Викишоп» с BERT, spaCy и TF-IDF

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Импорт библиотек

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline

from lightgbm import LGBMClassifier

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import numpy as np

import re

import time

import spacy

from tqdm.notebook import tqdm

import torch

import transformers

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Подготовка

In [3]:
data = pd.read_csv('toxic_comments.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Напишем функцию для обработки текста.

In [4]:
def clean_tokenize_lemmatize(text:str, stop_words:list=stopwords.words('english')) -> str:
    ''' Приведение к нижнему регистру -> фильрация символов совмещенная с токенизацией, остаются только латинские буквы и апострофы ->
    -> лемматизация -> в случае если на выход поступает пустая строка, она заменяется на '0'
    
    '''
    clean_tokens = []
    text = text.lower()
    tokens = re.sub(r'[^A-Za-z\']', ' ', text).split()
    for token in tokens:
        #if token not in stop_words:                                   # В списке стоп-слов нет некоторых стоп-слов в том виде в котором они приходят после лемматизации,
        clean_tokens.append(WordNetLemmatizer().lemmatize(token))      # поэтому лучше от них избавляться на этап раньше
    clean_string = ' '.join(clean_tokens)
    if clean_string == '': clean_string = '0'
    return clean_string                                   

P.s. были созданы версии лемматизатора с и без очистки от стоп-слов, однако тот, что стоп-слова не трогал позволил в конечном итоге получить чуть лучший результат f1.

И создадим новый столбец с лемматизированным и очищенным текстом.

In [5]:
%%time
data['lemmatized'] = data['text'].apply(clean_tokenize_lemmatize)

Wall time: 1min 4s


# Обучение

Обучение будет проводится при помощи трех подходов:

* С помощью TF-IDF
* С помощью библиотеки spaCy
* С помощью BERT

В каждом из подходов сгенерированные признаки будут поступать на вход следующему набору моделей:

In [6]:
models_params = {
    LogisticRegression(random_state=24) : dict(),
    LGBMClassifier(random_state=24) : dict(max_depth=range(-1, 201), num_leaves=range(5, 101), n_estimators=range(1, 201)),
    DummyClassifier() : dict(strategy=['most_frequent', 'prior', 'stratified', 'uniform', 'constant'])
}

В отличие от других подходов TF-IDF будет более разумно обучать через Pipeline. Для того чтобы TF-IDF векторизатор настраивался только на тренировочной выборке.

In [7]:
pipe_models_params = {
    LogisticRegression(random_state=24) : [{}],
    LGBMClassifier(random_state=24) : [{'model__max_depth' : range(-1, 201), 'model__num_leaves' : range(5, 101), 'model__n_estimators' : range(1, 201)}],
    DummyClassifier(strategy='uniform') : [{}]
}

Далее следуют функции для обучения и подбора гиперпараметров(`model_training`) и для отображения результатов(`model_scoring`)

In [8]:
def model_training(models_params, X, y, n_iter=10):
    trained_models=[]
    for model, params in models_params.items():
        timer = time.time()
        random_search = RandomizedSearchCV(model, 
                                           param_distributions=params, 
                                           scoring='f1', 
                                           random_state=24, 
                                           n_iter=n_iter, 
                                           cv=3, 
                                           verbose=True
                                          )
        random_search.fit(X, y)
        timer = time.time() - timer
        print(f'Обучение модели {model} заняло {timer:.2f} сек или {(timer / 60):.2f} мин')
        print('---------------------------------------------------------------------------------------')
        trained_models.append(random_search)
    return trained_models

In [9]:
def pipe_training(models_params, X, y, n_iter=10):
    trained_models=[]
    for model, params in models_params.items():
        timer = time.time()
        random_search = RandomizedSearchCV(Pipeline([('vectorizer', TfidfVectorizer()), ('model', model)]), 
                                           param_distributions=params, 
                                           scoring='f1', 
                                           random_state=24, 
                                           n_iter=n_iter, 
                                           cv=3, 
                                           verbose=True
                                          )
        random_search.fit(X, y)
        timer = time.time() - timer
        print(f'Обучение модели {model} заняло {timer:.2f} сек или {(timer / 60):.2f} мин')
        print('---------------------------------------------------------------------------------------')
        trained_models.append(random_search)
    return trained_models

In [10]:
def model_scoring(model, X_test, y_test): # функция для выведения оценки модели.
    print()
    print('Модель', model.best_estimator_)
    print('Лучший f1 при кросс-валидации =', model.best_score_)
    predicted = model.predict(X_test)
    print(f'f1 на тестовой выборке = {f1_score(y_test, predicted)}')
    print()
    return None

## TF-IDF

Создадим стратифицированные обучающую и тестовую выборки.

In [11]:
X = data['lemmatized']
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 ,stratify=y, random_state=24)

И обучим несколько моделей.

In [12]:
trained_models = pipe_training(pipe_models_params, X_train, y_train)

for model in trained_models:
    model_scoring(model, X_test, y_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Обучение модели LogisticRegression(random_state=24) заняло 54.31 сек или 0.91 мин
---------------------------------------------------------------------------------------
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Обучение модели LGBMClassifier(random_state=24) заняло 1068.51 сек или 17.81 мин
---------------------------------------------------------------------------------------
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Обучение модели DummyClassifier(strategy='uniform') заняло 32.71 сек или 0.55 мин
---------------------------------------------------------------------------------------

Модель Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model', LogisticRegression(random_state=24))])
Лучший f1 при кросс-валидации = 0.7307333387951501
f1 на тестовой выборке = 0.7484346224677717


Модель Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model',
       

Лучший результат на тестовой выборке показала модель LGBMClassifier.

## spaCy

Поскольку как spaCy так и BERT самостоятельно занимаются лемматизацией, то функция `clean_tokenize_lemmatize`, является не совсем подходящей. Поэтому определим функцию, которая делает то же самое что и `clean_tokenize_lemmatize` но без лемматизации.

In [13]:
def clean(text:str) -> str:
    clean_tokens = []
    text = text.lower()
    tokens = re.sub(r'[^A-Za-z\']', ' ', text).split()
    clean_string = ' '.join(tokens)
    if clean_string == '': clean_string = '0'
    return clean_string                 

In [14]:
data['clean_text'] = data['text'].apply(clean)

При испытании ядер spaCy для данной задачи лучшим оказалось среднее, так как маленькое давало плохой результат, а большое не давало прироста качества в сравнении со средним.

In [15]:
spacy_nlp = spacy.load('en_core_web_md')

Для того, чтобы преобразование текста в векторы проводилось не слишком долго в обучающую выборку возьмем только 30.000 значений. Это слегка отразится на качестве, но зато дело пойдет быстрее.

In [16]:
X = data['clean_text']
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=24)
X_train_sample, trash, y_train_sample, trash = train_test_split(X_train, y_train ,train_size=30000, stratify=y_train, random_state=24)
del trash

In [17]:
X_train_spacy = pd.DataFrame((spacy_nlp(lemma).vector for lemma in tqdm(X_train_sample.values)), index=y_train_sample.index)

  0%|          | 0/30000 [00:00<?, ?it/s]

In [18]:
X_test_spacy = pd.DataFrame((spacy_nlp(lemma).vector for lemma in tqdm(X_test.values)), index=y_test.index)

  0%|          | 0/31915 [00:00<?, ?it/s]

In [19]:
trained_models = model_training(models_params, X_train_spacy, y_train_sample, n_iter=30)

for model in trained_models:
    model_scoring(model, X_test_spacy, y_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Обучение модели LogisticRegression(random_state=24) заняло 3.59 сек или 0.06 мин
---------------------------------------------------------------------------------------
Fitting 3 folds for each of 30 candidates, totalling 90 fits
Обучение модели LGBMClassifier(random_state=24) заняло 450.61 сек или 7.51 мин
---------------------------------------------------------------------------------------
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Обучение модели DummyClassifier() заняло 0.49 сек или 0.01 мин
---------------------------------------------------------------------------------------

Модель LogisticRegression(random_state=24)
Лучший f1 при кросс-валидации = 0.6703808655286045
f1 на тестовой выборке = 0.6839811115147112


Модель LGBMClassifier(max_depth=48, n_estimators=198, num_leaves=35, random_state=24)
Лучший f1 при кросс-валидации = 0.708646808953095
f1 на тестовой выборке = 0.7184535528906391


Модель Dum

## BERT

Поскольку создание эмбеддингов в BERT занимает безбожно большое время в сравнении с приведенными выше способами, то выборка для которой будет применятся BERT будет довольно скромной (400 значений).

In [20]:
X_train_sample, trash, y_train_sample, trash = train_test_split(X_train, y_train ,train_size=400, stratify=y_train, random_state=24)
del trash

In [21]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")
X_train_tokenized = X_train_sample.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

Если не подать в `tokenizer.encode` параметры `max_length=512` и `truncation=True`, то может возникнуть проблема: *Token indices sequence length is longer than the specified maximum sequence length for this model (1084 > 512). Running this sequence through the model will result in indexing errors*.

В теории модель **XLNet** может справиться с более длинными чем 512 векторами. Но мне и так не ясно как эффективно применять BERT (особенно в рамках этого проекта), учитывая, что эмбеддинги производятся так долго. Так что ограничимся BERT.

Применение padding и создание маски:

In [22]:
max_len = 512
padded = np.array([i + [0]*(max_len - len(i)) for i in X_train_tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [23]:
config = transformers.BertConfig()
model = transformers.BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
batch_size = 100
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

features = pd.DataFrame(np.concatenate(embeddings), index=y_train_sample.index)

  0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
X_train_BERT, X_test_BERT, y_train_BERT, y_test_BERT = train_test_split(features, y_train_sample, test_size=0.5, stratify=y_train_sample, random_state=24)

trained_models = model_training(models_params, X_train_BERT, y_train_BERT, n_iter=100)

for model in trained_models:
    model_scoring(model, X_test_BERT, y_test_BERT)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Обучение модели LogisticRegression(random_state=24) заняло 0.26 сек или 0.00 мин
---------------------------------------------------------------------------------------
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Обучение модели LGBMClassifier(random_state=24) заняло 36.90 сек или 0.61 мин
---------------------------------------------------------------------------------------
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Обучение модели DummyClassifier() заняло 0.03 сек или 0.00 мин
---------------------------------------------------------------------------------------

Модель LogisticRegression(random_state=24)
Лучший f1 при кросс-валидации = 0.24074074074074073
f1 на тестовой выборке = 0.5625000000000001


Модель LGBMClassifier(max_depth=1, n_estimators=158, num_leaves=64, random_state=24)
Лучший f1 при кросс-валидации = 0.21666666666666667
f1 на тестовой выборке = 0.4999999999999999


Модель 

# Выводы

Лучшую метрику f1 на тестовой выборке приблизительно равную 0.78 показала модель LGBMClassifier, при генерации признаков с помощью TfidfVectorizer. Можно отметить, что такой способ обеспечивает довольно большую скорость генерации векторов, но в тоже время вызывает взрывной рост количества признаков (каждое новое слово = новый признак, в данном случае размер обучающей выборки составил 127656 строк, 139610 столбцов). Что плохо влияет на скорость обучения модели, поэтому медленно обучающиеся модели вместе с этим способом будут работать плохо.

---
При генерации векторов через библиотеку spaCy, аналогичные модели показали несколько худший результат на тестовой выборке (приблизительно 0.72 при модели LGBMClassifier). Кроме того для экономии времени модель была обучена лишь на 30.000 значений.

Хоть скорость генерации векторов и не так хороша как у TF-IDF, но в конечном итоге получается обучающая выборка с фиксированным количеством столбцов равным 300, что позволяет за приемлимое время обучать медленно обучаемые модели.

---

Лучший результат при генерации через BERT f1 = 0.56 на тестовой выборке при модели LogisticRegression. Такой низкий результат скорее всего обусловлен тем, что модель была обучена всего лишь на 200 значениях.

Быть может если обучить модель на эмбеддингах от полной обучающей выборки и проверить на эмбеддингах от полной тестовой выборки, то её метрика f1 превзошла бы все мои ожидания. Но я этого не узнаю поскольку эмбеддинг всего датафрейма занял бы приблизительно 55 часов на моей машине, а оно мне надо?